# Classification de ballons de football et de rugby

- Binôme: Hugo, Barreiro, Ahmed, Atmane
- Adresses mails: hugo.barreiro@universite-paris-saclay.fr / ahmed.atmane@universite-paris-saclay.fr
- [Dépôt GitLab](https://gitlab.dsi.universite-paris-saclay.fr/hugo.barreiro/L1InfoInitiationScienceDonnees-Semaine8)

Importation des différentes bibliotèques :

In [ ]:
import os, re
from glob import glob as ls
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()
from PIL import Image
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
from sys import path

from utilities import *
from intro_science_donnees import data
from intro_science_donnees import *

## Jeu de données

Chargement du jeu de données :

In [ ]:
dataset_dir = 'data'
images = load_images(dataset_dir, "*.png")

Visualisation du jeu de données :

In [ ]:
image_grid(images)

## Prétraitement

Nous allons prétraiter les images de notre jeu de données.

Pour ce faire nous allons créer une liste python pour ajouter au fur et à mesure nos nouvelles images. On la transformera en Series Pandas afin de créer un DataFrame.

Pour chaque image, nous allons appliquer un filtre de couleur, c'est-à-dire que l'on va récupérer un tableau en dimension 32x32 représentant une image en noir et blanc. Sur cette image, les zones blanches représenteront les zones où la couleur en question sera la plus forte, les zones noires représenteront les zones où la couleur en question sera la plus faible.

Ensuite, grâce à ces tableaux 32x32 nous pourrons créer des nouveaux tableaux en dimension 32x32x4. Pour ce faire, nous allons regarder pour chaque pixel des tableaux 32x32 si sa valeur est strictement supérieur à un taux fixé. Si c'est le cas, alors on fixe la couche RGBA à 255, 255, 255, 255 pour les tableaux 32x32x4. Sinon, on la fixe à 0, 0, 0, 0. Ainsi, nous pourrons récupérer des images où les objets seront en blanc alors que le fond sera transparent.

In [ ]:
#Création de la liste d'images prétraitées
images_clean = []


#Pink
images_clean.append(foreground_filter_taux(pinkness_filter(images[0]), 40))
images_clean.append(foreground_filter_taux(pinkness_filter(images[1]), 40))
images_clean.append(foreground_filter_taux(pinkness_filter(images[2]), 50))
images_clean.append(foreground_filter_taux(pinkness_filter(images[3]), 50))

#Yellow
images_clean.append(foreground_filter_taux(yellowness_filter(images[4]), 235))
images_clean.append(foreground_filter_taux(yellowness_filter(images[5]), 235))
images_clean.append(foreground_filter_taux(yellowness_filter(images[6]), 235))
images_clean.append(foreground_filter_taux(yellowness_filter(images[7]), 250))
images_clean.append(foreground_filter_taux(yellowness_filter(images[8]), 275))

#Green
images_clean.append(foreground_filter_taux(greenness_filter(images[9]), -125))

#Orange
images_clean.append(foreground_filter_taux(orangeness_filter(images[10]), -75))

#Brown
images_clean.append(foreground_filter_taux(brownness_filter(images[11]), -105))

#Orange
images_clean.append(foreground_filter_taux(orangeness_filter(images[12]), -25))
images_clean.append(foreground_filter_taux(orangeness_filter(images[13]), -50))
images_clean.append(foreground_filter_taux(orangeness_filter(images[14]), 15))
images_clean.append(foreground_filter_taux(orangeness_filter(images[15]), -40))
images_clean.append(foreground_filter_taux(orangeness_filter(images[16]), -50))

#Brown
images_clean.append(foreground_filter_taux(brownness_filter(images[17]), -20))

#Green
images_clean.append(foreground_filter_taux(greenness_filter(images[18]), -175))

#White
images_clean.append(foreground_filter_taux(whiteness_filter(images[19]), 325))


#Création d'une Series Pandas à partir de la liste python d'images prétraitées
images_clean = pd.Series(images_clean, copy=True)


#Visualisation
image_grid(images_clean)

Nous allons maintenant créer un DataFrame afin de traiter les images et classer :

In [ ]:
columns = ['Elongation', 'Matched_filter', 'Class']

index = images.index

liste_Elongation = [elongation_matrice(img) for img in images_clean]
liste_MF = [matched_filter_matrice(img, images_clean) for img in images_clean]
liste_Class = [1 for i in range(10)] + [-1 for i in range(10)]

data = {'Elongation' : liste_Elongation, 'Matched_filter' : liste_MF, 'Class' : liste_Class}

df= pd.DataFrame(data=data, index=index, columns=columns)
df

Nous allons à présent standariser le DataFrame afin d'étudier et analyser correctement les données.

In [ ]:
dfstd = (df - df.mean()) / df.std()
dfstd["Class"] = df["Class"]
dfstd

## Visualisation des données

Affichons les statistiques de notre DataFrame :

In [ ]:
dfstd.describe()

Nous remarquons que la table est bien standarisée : l'écart-type est de 1 et la moyenne est de 0.

Affichons une carte de chaleur afin de mieux visualiser nos données :

In [ ]:
dfstd.style.background_gradient(cmap='coolwarm')

On remarque que le Matched_Filter n'a pas correctement fonctionné sur 2 images et que l'Elongation n'a pas donné un résultat satisfaisant pour 1 image.

Affichons la matrice de corrélation sous forme de carte de chaleur :

In [ ]:
dfstd.corr().style.background_gradient(cmap='coolwarm')

On remarque que l'élongation corrèle plus avec les ballons de rugby et que le matched_filter corrèle plus avec les ballons de football.

Regardons si nos données sont bien réparties grâce aux différents attributs :

In [ ]:
make_scatter_plot(dfstd, images_clean, axis='square')

Avec les images prétraitées, nous ne voyons pas grand chose.

Affichons la répartition avec les images de base :

In [ ]:
make_scatter_plot(dfstd, images, axis='square')

La plupart des ballons de football et de rugby sont correctement répartis. Néanmoins, 2 ballons de football et 1 ballon de rugby ne sont pas correctement répartis (Cela correspond en effet aux erreurs détectés lors de la visualisation des données). Des erreurs risques de se produire lors de la classification.

## Classificateurs favoris

Nous allons tester plusieurs classificateurs différents, utilisant des méthodes de classification différentes afin de trouver celui qui fonctionne le mieux avec nos données.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model_name = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
model_list = [
    KNeighborsClassifier(1),
    SVC(kernel="linear", C=0.025, probability=True),
    SVC(gamma=2, C=1, probability=True),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

Affichons les résultats :

In [ ]:
from sklearn.metrics import balanced_accuracy_score as sklearn_metric
compar_results = systematic_model_experiment(df, model_name, model_list, sklearn_metric)
compar_results.style.set_precision(2).background_gradient(cmap='Blues')

Affichons les résultats de manière plus visuel :

In [ ]:
compar_results[['perf_tr', 'perf_te']].plot.bar()
plt.ylim(0.5, 1)
plt.ylabel(sklearn_metric.__name__);

Nous remarquons que tous les classificateurs nous donne d'excellents résultats.

Regardons quel est le plus efficace :

In [ ]:
model_list[compar_results.perf_te.argmax()]

Le Knn est le meilleur classificateur dans notre configuration.

Nous avons décidé d'utiliser qu'un seul voisin car après des tests pour plusieurs valeurs, c'est celle là qui obtenait les meilleurs résultats.

## Résultats

### Observations

Nous allons à présent observer les résultats obtenus.

Nous remarquons qu'avec le Knn nous obtenons un taux de réussite de 94% avec un taux d'erreur de 7%. Ce qui est un résultat plus que satisfaisant à notre niveau.

En effet, classifier notre jeu de données pouvaient de montrer un peu compliqué. Une balle de rugby vu de face ressemble grandement à une balle de football. Enlever le fond de certaines images pouvait être délicat : fond non-uni, ombres, perspectives, accessoires par exemple.

On observe 2 erreurs sur le matched_filter : 2 ballons de football sont confondus avec des ballons de rugby.

On observe 1 résultat non-désiré sur l'élongation : 1 ballon de rugby n'est pas allongé.

### Interprétations

Le Knn est surement plus efficace avec un seul voisin puisque les ballons de football mals répartis devaient être identifiés en ballons de rugby avec plus de voisins. En effet, ils sont plus proches de balles de rugby que des balles de football.

Les erreurs du matched_filter sont probablement du au prétraitement des 2 ballons de football en question : En effet, les motifs assez présent sur les balles n'ont pas été pris en compte dans l'image finale. Ainsi, leur forme n'était pas parfaitement ronde.

Le résultat non désiré pour l'élongation est probablement du à la forme assez ronde du ballon de rugby vu de face. Ainsi, ne voyant pas le côté du ballon, on dirait que celui-ci est rond et donc non allongé.

## Discussion

Nos données peuvent potentiellement comporter des biais.

En effet, plusieurs ballons sont de la même couleur : rose, jaune, marron ou orange. Cela peut poser problème si l'on s'était basé sur la couleur des balles pour les classer. Ce n'est pas le cas, ainsi ce biai ne devrait pas impacter notre jeu de données.

Ensuite, nous avons peut-être négliger des positions potentielles pour les ballons de rugby ou de football. Ce biai peut nous impacter car on utilise matched_filter. Avoir différentes positions impacte le template. Si des positions ont été occultées, alors le template n'est pas totalement représentatif. De plus, si certaines positions sont sur-représenter alors le template n'est également pas totalement représentatif. Ce biai doit nous impacter puisque certaines positions de balles de rugby sont plus représentées que d'autres.

Enfin, un dernier biai nous concerne au niveau de l'élongation. On est parti du principe qu'un ballon de rugby est allongé. Ce n'est pas forcément le cas sous tous les angles. En effet, vu de face la balle de rugby est ronde. Un biai est donc présent dans notre analyse.

Notre méthode de prétraitement n'est pas appliquable telle quelle sur un jeu de données largement plus grand. En effet, nous appliquons un prétraitement personnalisé à chaque balle en fonction de sa couleur. Or, si un jeu de données possède 50, 100 ou encore 1000 images, nous ne pouvons pas appliqué un prétraitement personnalisé pour chaque image comme nous l'avons fait.

Pour remédier à ce problème nous pourrions imaginer une fonction qui calcul le taux de couleur d'une image pour plusieurs couleurs et qui renvoie quelle est la couleur la présente dans l'image. Les images étant bien centrée sur la balle, la couleur caractéristique du ballon sera la plus présente. Enfin, sachant la couleur la plus présente, donc celle de la balle, nous pourrions appeler le filtre de couleur adapté pour chaque image. Pour ce qui est du taux pour le foreground_filter nous pourrions tester plusieurs valeurs et choisir une valeur moyenne qui convienne à un près pour chaque image.

Nous avons rencontrés des problèmes notamment par rapport au format des images, des différentes couches RGBA et également pour extraire correctement les ballons du fond.

Les images étant au format JPG elles ne disposaient pas de couche de transparence. Ainsi, nous avons du la rajouter lors du traitement des images afin d'utiliser correctement différentes fonctions comme élongation qui se base grandement sur la couche de transparence.

Pour extraire les ballons du fond, nous avons du coder plusieurs fonctions de filtre pour les différentes couleurs et ensuite tester différentes valeurs pour le taux afin d'extraire correctement les objets. Ceci est un peu laborieux mais on à finalement réussi à s'en sortir.

## Conclusion

Pour conclure, ce projet de classification de ballons de football et de rugby nous a permis d'apprendre à extraire des objets de fonds non-uni, avec des ombres, de la perspective et des accessoirs afin de classer nos images.

Cela nous a également appris à manier les images avec différents format et couches internes.

Enfin, ce projet et cette UE de manière générale nous ont appris à s'informer, à trouver des solutions par nous-mêmes et à travailler en autonomie mais également en équipe.